# Introduction

In order to successful knowledge discovery in databases (KDD), well-defined and formal methods should be applied for managing data.  Cross-industry standard process for data mining (CRISP-DM) model is a standard methodology, which includes six phases:
    1. Problem domain understanding
    2. Data Understanding
    3. Data Preparation
    4. Modeling
    5. Evaluation
    6. Deployment

----------------------------------------------------------------------------------------------------------------------

# Part 1 - Problem domain understanding

Cancer still remains a challenge for our world in preventing and treating. However, most of cancers are highly curable if they are detected early, so the stage at diagnosis heavily influences survival. Due to no early warning signs, it’s important to have routine screening tests. For many types of cancers such as colorectal cancer, lung cancer, stomach cancer, screening rate remains low due to unpleasant procedure and expensive cost. Therefore, a risk prediction model for cancer could bring benefits for both customer and health institute. For customer, it encourages people to take screening tests to detect the risk of cancer early and increase survival rate. For health institute, it provides more services and hence increase sale.

Nowadays, electronic medical records have become increasingly available through regular health checkup. In recent research, there has been an increasing interest in finding biomarkers of cancer from routine blood tests. In general, blood indices are related to cancer to some extent, but none of them solely exhibits a clear connection and can be used for diagnostic purposes. However, taking these basic blood indices together, information to be gleaned may reveal about converging signs or pattern of an individual for many forms of cancer. By monitoring selected biomarkers routinely measured in primary care, we can learn a lot about physiological patterns that promote carcinogenesis, proliferation, progression before tumor makers emerge.

This research aims to utilize temporal, longitudinal data accumulated in regular health checkup to explore pattern of change of many biomarkers in common blood test to predict cancer.

----------------------------------------------------------------------------------------------------------------------

# Part 2 - Data Exploration & Understanding

http://localhost:8888/notebooks/0-MyCollection/00-Sample/01-DataExploration.ipynb

## 1. Import Library and Define Common function

### 1.1. Import Library

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pylab
from scipy import stats
from sklearn.linear_model import LinearRegression
import seaborn as sns
sns.set()

# Modelling Helpers:
# from sklearn.preprocessing import Imputer, Normalizer, scale
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, ShuffleSplit, cross_validate
from sklearn import model_selection
from sklearn.model_selection import train_test_split

# Classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from lightgbm import LGBMClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
# Evaluation metrics for Classification
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from sklearn.metrics import mutual_info_score

# Regression
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV,ElasticNet,LogisticRegression
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
# Evaluation metrics for Regression 
from sklearn.metrics import mean_squared_log_error, mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import (confusion_matrix, classification_report, accuracy_score, roc_auc_score, auc,
                             precision_score, recall_score, roc_curve, precision_recall_curve,
                             precision_recall_fscore_support, f1_score,
                             precision_recall_fscore_support)

# Configuration
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


# Supress warnings
import warnings
warnings.filterwarnings("ignore")

print("Setup complete...")

Setup complete...


### 1.2. Common Function

In [2]:
# Distribution plot

def analyse_continuous(df,var,target):
    df = df.copy()
    # df[var] = df[var].fillna(df[var].median())
    plt.figure(figsize=(20,5))
       
    # histogram
    plt.subplot(131)
    sns.distplot(df[var], bins=30)
    #sns.distplot(df[var],hist=True, kde=True,kde_kws={'shade': True, 'linewidth': 3})
    plt.title('Histogram')    
    
    # Q-Q plot
    plt.subplot(132)
    stats.probplot(df[var], dist="norm", plot=pylab)
    plt.ylabel('Quantiles')    
    
    # boxplot
    plt.subplot(133)
    sns.boxplot(x=df[var])
    plt.title('Boxplot')
          
    # skewness and kurtosis
    print('Skewness: %f' % df[var].skew())
    print('Kurtosis: %f' % df[var].kurt())
    plt.show()

In [3]:
def Training_Preparation(df, cont_vars):
    num_df = df[cont_vars].copy()

    # scaling features
    from sklearn.preprocessing import MinMaxScaler
    numdf_norm = pd.DataFrame(MinMaxScaler().fit_transform(df[cont_vars]))
    numdf_norm.columns = num_df.columns
    
    # Define X & y
    X = numdf_norm
    y = df['Class']

    # Split to train and test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=90, stratify = y)
    
    # initialize models
    models = []
    models.append(('KNN', KNeighborsClassifier()))
    models.append(('SVC', SVC(kernel="linear")))
    models.append(('LSVC', SVC(kernel="rbf")))
    models.append(('LR', LogisticRegression()))
    models.append(('DT', DecisionTreeClassifier()))
    models.append(('GNB', GaussianNB()))
    models.append(('RF', RandomForestClassifier()))
    models.append(('GB', GradientBoostingClassifier()))
    models.append(('LGB',LGBMClassifier()))
    models.append(('ADA',AdaBoostClassifier()))
    models.append(('LDA',LinearDiscriminantAnalysis()))
    models.append(('QDA',QuadraticDiscriminantAnalysis()))
    models.append(('NN',MLPClassifier()))
    models.append(('XGB',XGBClassifier()))
    
    # Test options and evaluation metric
    seed = 9
    scoring = 'recall_macro'

    # evaluate each model in turn
    results = {}
    names = []

    for name, model in models:
        kfold = model_selection.KFold(n_splits=10, random_state = seed)
        cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
        results[name] = cv_results
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
        
    results_df = pd.DataFrame(results)
    plt.figure(figsize=(16,8))
    sns.boxplot(data=results_df)
    plt.show()

In [4]:
from matplotlib.backends.backend_pdf import PdfPages


def DistributionComparison(all_df, selected_vars,name):
    colors = ['#3791D7','#D72626']

    # pdf = matplotlib.backends.backend_pdf.PdfPages(name + '.pdf')
    with PdfPages(name + '.pdf') as pdf_pages:
        for column in selected_vars:    
            fig = plt.figure(figsize=[8,4])
            plt.subplot(121)
            sns.boxplot(x='Class', y=column,data=all_df,palette=colors)
            plt.title(column, fontsize=12)
            plt.subplot(122)
            sns.kdeplot(all_df[all_df.Class==1][column], bw = 0.4, label = "Cancer", shade=True, color="#D72626", linestyle="--")
            sns.kdeplot(all_df[all_df.Class==0][column], bw = 0.4, label = "NoCancer", shade=True, color= "#3791D7", linestyle=":")
            plt.title(column, fontsize=12)   
            pdf_pages.savefig(fig)                                          
            plt.show()    

    # Write the PDF document to the disk
    #pdf_pages.close()

In [5]:
def ModelEvaluation(df, cont_vars):
    
    num_df = df[cont_vars].copy()

    # scaling features
    from sklearn.preprocessing import MinMaxScaler
    numdf_norm = pd.DataFrame(MinMaxScaler().fit_transform(df[cont_vars]))
    numdf_norm.columns = num_df.columns
    
    # Define X & y
    X = numdf_norm
    y = df['Class']

    # Split to train and test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=90, stratify = y)
    
    # initialize models
    models = []
    models.append(('KNN', KNeighborsClassifier()))
    models.append(('SVC', SVC(kernel="linear")))
    models.append(('LSVC', SVC(kernel="rbf")))
    models.append(('LR', LogisticRegression()))
    models.append(('DT', DecisionTreeClassifier()))
    models.append(('GNB', GaussianNB()))
    models.append(('RF', RandomForestClassifier()))
    models.append(('GB', GradientBoostingClassifier()))
    models.append(('LGB',LGBMClassifier()))
    models.append(('ADA',AdaBoostClassifier()))
    models.append(('LDA',LinearDiscriminantAnalysis()))
    models.append(('QDA',QuadraticDiscriminantAnalysis()))
    models.append(('NN',MLPClassifier()))
    models.append(('XGB',XGBClassifier()))
    
    for name,model in models:
        print(name)
        model.fit(X_train, y_train)
        
        print('==========================================================')
        print('Train set')
        y_train_pred = model.predict(X_train)
        print('Accuracy: ', accuracy_score(y_train, list(y_train_pred)))
        print('ROC AUC Score: ', roc_auc_score(y_train, list(y_train_pred)))
        cm_df = pd.DataFrame(confusion_matrix(y_train,list(y_train_pred)), index=model.classes_,columns=model.classes_)
        cm_df.index.name = 'True'
        cm_df.columns.name = 'Predicted'
        print('Confusion matrix')
        print(cm_df)
        print(classification_report(y_train, list(y_train_pred)))
  
        print('----------------------------------------------------------')
        print('Test set')
        y_test_pred = model.predict(X_test)
        print('Accuracy: ', accuracy_score(y_test, list(y_test_pred)))
        print('ROC AUC Score: ', roc_auc_score(y_test, list(y_test_pred)))
        cm_df = pd.DataFrame(confusion_matrix(y_test,list(y_test_pred)), index=model.classes_,columns=model.classes_)
        cm_df.index.name = 'True'
        cm_df.columns.name = 'Predicted'
        print('Confusion matrix')
        print(cm_df)
        print(classification_report(y_test, list(y_test_pred)))
        print('==========================================================')
        

In [6]:
from sklearn.manifold import TSNE

def tsne_plot(X, y):
       
        
    # scaling features
    from sklearn.preprocessing import MinMaxScaler
    numdf_norm = pd.DataFrame(MinMaxScaler().fit_transform(X))
    numdf_norm.columns = X.columns
    
    tsne = TSNE(n_components=2, random_state=0)
    X_t = tsne.fit_transform(numdf_norm)

    plt.figure(figsize=(12, 8))
    plt.scatter(X_t[np.where(y == 0), 0], X_t[np.where(y == 0), 1], marker='o', color='g', linewidth='1', alpha=0.8, label='No cancer')
    plt.scatter(X_t[np.where(y == 1), 0], X_t[np.where(y == 1), 1], marker='o', color='r', linewidth='1', alpha=0.8, label='Colon cancer')

    plt.legend(loc='best');
    plt.show();

In [7]:
# function to find upper and lower boundaries
# for normally distributed variables

def find_normal_boundaries(df, variable):

    # calculate the boundaries outside which sit the outliers
    # for a Gaussian distribution

    upper_boundary = df[variable].mean() + 3 * df[variable].std()
    lower_boundary = df[variable].mean() - 3 * df[variable].std()

    return upper_boundary, lower_boundary

In [8]:
# function to find upper and lower boundaries
# for skewed distributed variables

def find_skewed_boundaries(df, variable, distance):

    # Let's calculate the boundaries outside which sit the outliers
    # for skewed distributions

    # distance passed as an argument, gives us the option to
    # estimate 1.5 times or 3 times the IQR to calculate
    # the boundaries.

    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)

    lower_boundary = df[variable].quantile(0.25) - (IQR * distance)
    upper_boundary = df[variable].quantile(0.75) + (IQR * distance)

    return upper_boundary, lower_boundary

In [9]:
def find_uncorrelated_vars(cancer_df, selected_vars, threshold):

    corrmat = cancer_df[selected_vars].corr()
    corrmat = corrmat.abs().unstack() # absolute value of corr coef
    corrmat = corrmat.sort_values(ascending=False)

    corrmat = pd.DataFrame(corrmat).reset_index()
    corrmat.columns = ['feature1', 'feature2', 'corr']
    corrmat['MissingF1'] = corrmat.feature1.apply(lambda x:MissingPercentage(x))
    corrmat['MissingF2'] = corrmat.feature2.apply(lambda x:MissingPercentage(x))
    
    correlated_groups = corrmat[corrmat['corr'] > threshold]
    
    selected_vars = []
    remaining_vars = correlated_groups.feature1.unique()

    while(len(remaining_vars) > 0):
        feature = remaining_vars[0]
        correlated_block = correlated_groups[correlated_groups.feature1 == feature]
        min_ind = correlated_block[['MissingF2']].idxmin() 
        sel_var = correlated_block.feature2[min_ind].values[0]
        removed_vars = [var for var in list(correlated_block.feature2.values)]
        remaining_vars = [var for var in remaining_vars if var not in removed_vars]
        if sel_var not in selected_vars:
            selected_vars = selected_vars + [sel_var]   
    
    return selected_vars

In [10]:
def analyze_na_values(df, var, target):
    tmp_df = df.copy()
    print(target)
    
    # Make a variable that indicates 1 if the observation was missing or 0 otherwise
    tmp_df['Missing'] = np.where(df[var].isnull(),1,0)
    
    # Calculate the mean Price where the information is missing or present
    tmp_df =  pd.DataFrame(tmp_df.groupby([target,'Missing'])[target].count())
    tmp_df.columns = ['Count']
    tmp_df = tmp_df.reset_index()
    
    
    if(len(tmp_df[tmp_df[target] == 0] == 1)):
        tmp_df= tmp_df.append({target:0,'Missing':1,'Count':0}, ignore_index=True)

    if(len(tmp_df[tmp_df[target] == 1] == 1)):
        tmp_df= tmp_df.append({target:1,'Missing':1,'Count':0}, ignore_index=True)

    tmp_df.loc[0,'Per']= tmp_df.loc[0,'Count']/(tmp_df.loc[0:1,'Count'].sum())
    tmp_df.loc[1,'Per']= tmp_df.loc[1,'Count']/(tmp_df.loc[0:1,'Count'].sum())
    tmp_df.loc[2,'Per']= tmp_df.loc[2,'Count']/(tmp_df.loc[2:3,'Count'].sum())
    tmp_df.loc[3,'Per']= tmp_df.loc[3,'Count']/(tmp_df.loc[2:3,'Count'].sum())
    sns.barplot(x=target, y = 'Per', data=tmp_df, hue='Missing')
    plt.title(var)
    plt.show()

In [11]:
def CategoricalDistribution(df, var, target):
    df = df.copy()
    
    # Calculate the mean Price where the information is missing or present
    sns.countplot(x=var, data=df, hue=target)
    plt.title(var)
    plt.show()

In [12]:
def CreateDummyVar(df, categorical_list):
    objdf_new = df.copy()
    objdf_dummy =pd.DataFrame()
    i = 0
    for e in categorical_list:
        i = i + 1
        objdf_new[e] = e + '_' + objdf_new[e].astype(str)
        varname= e 
        df_temp = pd.get_dummies(objdf_new[varname], drop_first=True)
        objdf_dummy = pd.concat([objdf_dummy, df_temp], axis=1)
        
    return objdf_dummy

In [13]:
def MissingPercentage(x):
    return df[x].isnull().sum()/len(df)

## 2. Load raw data

In [207]:
df = pd.read_csv('NHANES-MultipleCycles_merged1.csv')
org_df = df.copy()

df = df[df.RIAGENDR==2]
print(df.shape)

df.head()

(35481, 799)


,SIALANG,WTINT2YR,DMDCITZN,WTMEC2YR,SIAINTRP,SIAPROXY,RIDAGEYR,DMDMARTL,RIDSTATR,MIALANG,FIAPROXY,SDMVPSU,MIAPROXY,RIAGENDR,DMDEDUC2,FIALANG,SEQN,SDMVSTRA,RIDAGEMN,RIDEXPRG,MIAINTRP,DMDYRSUS,DMDEDUC3,DMDHHSIZ,DMDFMSIZ,DMDHRGND,RIDEXMON,RIDRETH1,SDDSRVYR,INDFMPIR,FIAINTRP,MCQ160D,BPQ070,ACD040,MCQ025,IMQ011,PFQ061T,ALQ130,FSDAD,CDQ006,DPQ060,SMQ050U,DEQ034A,HUQ020,SMD100NI,DLQ010,DID250,PFQ061F,HSQ580,DIQ230,DPQ040,DPQ020,SXQ610,INQ012,SMQ710,FSD071,SXQ600,SMQ876,CDQ005,PFQ030,DEQ034C,DEQ038G,INQ020,RHD280,SMQ800,CDQ001,SMAQUEX2,HIQ260,PFQ061I,MCQ050,DLQ050,RHQ602U,FSD151,HSQ500,FSD111,HUQ071,PFQ051,SMQ690F,DIQ350U,SMQ870,HIQ031B,INQ150,SMQ670,HIQ210,CDQ009A,MCQ160L,SMD100TR,SXQ590,PFQ041,PFQ061Q,SMDUPCA,BPQ100D,HIQ031A,SMQ856,MCQ160A,SMQ690D,HSQ571,DLQ040,DIQ180,SXD171,SXQ130,OCQ260,SMQ868,FSD032A,FSD032E,PFQ063D,FSDHH,SXQ251,FSQ162,DPQ050,BPQ080,SXQ270,OCQ210,CDQ009H,INQ090,SMD057,INDFMMPI,RHQ060,PFQ061S,SMD100FL,BPQ030,RHD143,RHQ420,HIQ011,SMQ690B,DPQ070,SMQ020,FSDCH,HSD010,SMQ050Q,SXD031,RHQ171,HIQ031I,RHQ291,DEQ034D,PFQ020,PFQ061D,MCQ053,SMQ880,DPQ100,HIQ031C,SMD100MN,MCQ230B,ALQ110,RHQ586U,MCQ160F,OCD270,SMQ690C,ALQ101,MCQ170L,FSD102,FSD032F,OCQ380,INQ140,RHQ166,OCD390G,HEQ040,SMQ770,CDQ009D,PUQ110,RHQ020,RHQ169,BPQ090D,MCQ080,SXQ265,OCD395,DLQ020,SLQ050,MCQ160M,DIQ080,PFQ061A,RHQ070,RHQ010,HIQ031AA,SMQ720,INQ060,RHQ131,BPQ020,HUQ030,BPQ040A,SMD100LN,DID060,PFQ049,DED031,ECD010,SMQ817,HSAQUEX,HIQ031F,MCQ300B,CDQ008,MCQ300A,INDFMMPC,DIQ360,DIQ050,PFQ061N,PUQ100,RHQ031,DLQ060,SMD641,FSD061,SMQ040,ECQ150,SMQ690A,FSD032C,PFQ090,DIQ010,IND235,RHQ576U,FSD122,PFQ061M,DLQ080,RHQ602Q,MCQ160G,DIQ260U,SMQ725,FSD032B,FSD052,PFQ061P,FSD146,DPQ080,HSQ590,RHQ160,PFQ061K,PFQ061C,PFQ063A,PFQ063E,MCQ160E,WHQ030E,ALQ120U,OCD150,MCQ230D,HEQ030,FSD132,ECQ090,DIQ170,PFQ061J,PFQ061O,CDQ009G,MCQ220,MCQ040,MCQ230A,SMQ840,SMQ872,DID350,MCQ080E,DID260,FSD141,DPQ030,HIQ031J,DIQ060U,RHQ596,SMQ874,MCQ149,MCQ300C,MCQ160C,SXQ272,RHQ554,FSD081,SMQ740,SXQ410,SXQ280,CDQ009C,SMD100CO,RHQ560U,SMQ860,SMD030,PFQ061R,RHQ576Q,MCQ230C,HIQ031E,RHQ540,HUQ090,CDQ003,MCQ092,PFQ063C,PFQ057,IMQ020,HSQ510,SXQ550,RHQ560Q,MCQ160K,SMD650,INQ030,INQ132,DID040,ECQ020,RHQ570,CDQ010,HEQ010,DPQ090,MCQ170M,PFQ061H,SMQ830,RHQ200,HUD080,ALQ120Q,BPQ050A,FSD032D,MCD093,PFQ063B,SMQ878,HEQ020,MCQ170K,CDQ002,MCQ010,SMQ858,PFQ061G,MCQ035,SMQ690E,ECQ080,HIQ105,HUQ010,RHQ580,FSD092,CDQ009F,SMD100BR,SMD630,PFQ059,PFQ061B,CDQ009B,CDQ004,CDQ009E,DPQ010,ECD070A,ECD070B,DIQ240,SXQ490,HSQ520,FSD041,BPQ060,HIQ270,INQ080,PFQ061L,PFQ054,SMD093,PFQ061E,HIQ031H,SXQ260,HIQ031D,SMQ866,OCQ180,MCQ160B,DIQ160,SMQ862,RHQ586Q,LBDSGLSI,URDTIME2,LBDSBUSI,LBDHDDSI,URDMNPLC,LBXHCG,LBDLYMNO,LBXSCA,PHQ060,URXMBP,LBDSCASI,URXMZP,URDFLOW2,ORXH51,ORXH64,LBXGLU,ORXHPC,LBXBGE,LBDHEG,LBXBPB,LBDWFL,LBDBCDSI,ORXGL,ORXH69,URXUMS,LBXSGL,LBXSOSSI,LBXIHG,URXMHH,LBDSALSI,URXVOL3,PHQ050,URXMHP,LBDHD,LBXHCR,ORXH62,URDTIME1,URDCNPLC,LBXSZN,LBXSKSI,LBDTHGSI,URXMIB,URDFLOW3,ORXHPV,URXMNP,LBXBCD,LBXSATSI,LBXSBU,ORXH11,LBDSTRSI,LBDSCRSI,ORXH73,LBXSUA,URXUTRI,ORXH26,ORXH31,LBDHBG,LBDSTPSI,PHASUPHR,ORXH82,LBXHE1,LBXSTR,ORXH53,ORXH58,URXVOL1,LBXLYPCT,URXVOL2,LBDSTBSI,LBDHEM,URXECP,LBDTCSI,LBXSCLSI,LBXSNASI,URDMZPLC,LBXSAL,PHAGUMHR,LBXNEPCT,PHQ020,LBXBGM,LBXHGB,ORXH35,ORXH84,LBXSTP,LBDIHGLC,URXCNP,LBXPLTSI,LBDIHGSI,URXPREG,LBDSIRSI,PHQ040,LBXSGTSI,LBDBGELC,PHAANTHR,ORXH68,LBXBAPCT,URDMEPLC,LBXSTB,ORXH81,LBXMCHSI,ORXGH,LBDNENO,LBXSCR,ORXH52,URDMBPLC,LBDSZNSI,ORXH56,LBXHE2,LBXMOPCT,LBXMPSI,ORXH61,LBDBGMLC,LBXSIR,PHAANTMN,PHACOFHR,URDECPLC,URDMOHLC,ORXH33,ORXH06,ORXHPI,LBDTHGLC,LBDSCUSI,LBDMONO,PHAALCMN,LBXRBCSI,URDMHPLC,URXUIO,LBDSUASI,LBDSPHSI,URDCOPLC,URXCRS,URDMC1LC,URXMEP,ORXH45,ORXH55,LBXMCVSI,ORXH71,LBXSCH,URXMC1,LBXGLT,LBDEONO,PHQ030,ORXH40,ORXH39,ORXH83,ORXH66,LBXHBC,ORXH72,WTSVS2YR,LBXSLDSI,LBXRDW,WTFSM,ORXH54,PHAGUMMN,ORXH70,LBXSGB,LBDBPBSI,URXUCL,LBDGLTSI,LBDWFLLC,WTSVOC2Y,URXCOP,LBDGLUSI,LBXSCU,LBXSSE,LBXWBCSI,LBDHDD,LBDSSESI,PHAALCHR,URDFLOW1,LBXHA,ORXH59,LBXSC3SI,LBXSPH,PHACOFMN,PHASUPMN,ORXH67,WTSOG2YR,ORXH42,URXUMA,LBXSAS

## 4. Target variable analysis

In [208]:
df[(df.MCQ230C.isna()== False) | (df.MCQ230D.isna()== False)].shape

(26, 799)

In [209]:
df = df[(df.MCQ230C.isna()) & (df.MCQ230D.isna())]
df.shape

(35455, 799)

In [210]:
df[(df['MCQ230A'] != 14) & (df['MCQ230B'] != 14) & (df.MCQ220 == 1)].shape

(1376, 799)

In [211]:
df = df[(df['MCQ230A'] == 14) | (df['MCQ230B'] == 14) | (df.MCQ220 == 2)]
df.shape

(19079, 799)

In [212]:
# no cancer group, excellent health
df['Class'] == 0

# breast cancer
df.loc[df.MCQ220 == 1,'Class'] = 1

df[df.Class == 1].shape[0]/df.shape[0],df[df.Class == 1].shape[0],df[df.Class == 0].shape[0]

(0.03165784370250013, 604, 18475)

In [213]:
# number of cancer <= 45 and num of cancer > 45
(df[(df.Class == 1) & (df.RIDAGEYR <= 45)].shape, df[(df.Class == 1) & (df.RIDAGEYR > 45)].shape)

((27, 799), (577, 799))

In [214]:
# number of no cancer <= 45 and num of no cancer > 45
(df[(df.Class == 0) & (df.RIDAGEYR <= 45)].shape, df[(df.Class == 0) & (df.RIDAGEYR > 45)].shape)

((8891, 799), (9584, 799))

In [215]:
df = df[df.RIDAGEYR > 45]
df.shape

(10161, 799)

In [216]:
pd.DataFrame([df[df.Class == 0].RIDAGEYR.describe(),df[df.Class == 1].RIDAGEYR.describe()],index=['NoCancer','Cancer'])

,count,mean,std,min,25%,50%,75%,max
NoCancer,9584.0,62.505426,10.700078,46.0,53.0,62.0,71.0,85.0
Cancer,577.0,68.532062,10.125914,46.0,61.0,70.0,79.0,85.0


In [217]:
df[df.Class == 1].shape[0]/df.shape[0],df[df.Class == 1].shape[0],df[df.Class == 0].shape[0]

(0.05678574943411081, 577, 9584)

In [218]:
df = df.reset_index(drop = True)
df.shape

(10161, 799)

## 5. Categorize vars

In [219]:
target = ['Class']

cont_vars = ['RIDAGEYR', 'RIDAGEMN', 'INDFMPIR', 'LBXSAL', 'LBDSGLSI', 'URDTIME2', 'LBDSBUSI', 'LBDHDDSI', 'LBDLYMNO', 'LBXSCA', 'URXMBP', 'LBDSCASI', 'URXMZP', 'URDFLOW2', 'LBXGLU', 'LBXBPB', 'LBDBCDSI', 'URXUMS', 'LBXSGL', 'LBXSOSSI', 'URXUAC', 'LBXIHG', 'URXMHH', 'URXMHP', 'URXUBA', 'WTFSM', 'URXUUR', 'URDTIME1', 'URXUSR', 'LBXSZN', 'URXUTU', 'LBXSKSI', 'URXUSN', 'URXUMMA', 'LBDTHGSI', 'URXMIB', 'URXUMO', 'URXMNP', 'LBXBCD', 'URXUDMA', 'LBXSATSI', 'LBXSBU', 'LBXV4C', 'LBDSTRSI', 'LBXVBZ', 'LBDSCRSI', 'LBXSUA', 'LBXVOX', 'URXUAS', 'URXUSB', 'LBDSTPSI', 'LBXVXY', 'LBXSTR', 'WTSA2YR', 'URXVOL1', 'LBXLYPCT', 'URXUTL', 'URXUPB', 'URXVOL2', 'PHAFSTMN', 'LBXVEB', 'URXECP', 'LBDTCSI', 'LBXNEPCT', 'LBXBGM', 'LBXHGB', 'LBXSTP', 'URXCNP', 'LBXPLTSI', 'LBDIHGSI', 'LBDSIRSI', 'LBXSGTSI', 'URXUCO', 'LBXBAPCT', 'LBXMCHSI', 'LBDNENO', 'LBXSCR', 'LBDSZNSI', 'LBXMOPCT', 'LBXMPSI', 'LBXVDB', 'LBXSIR', 'URXUCS', 'LBX2DF', 'URXUAB', 'LBDSCUSI', 'LBXRBCSI', 'URXUIO', 'LBDSUASI', 'LBDSPHSI', 'URXCRS', 'URXMEP', 'URXUCD', 'LBXMCVSI', 'URXUAS3', 'LBXSCH', 'URXMC1', 'LBXGLT', 'WTSVS2YR', 'LBXSLDSI', 'LBXRDW', 'PHAGUMMN', 'LBXSGB', 'LBDBPBSI', 'LBDGLTSI', 'WTSVOC2Y', 'URXCOP', 'LBDGLUSI', 'LBXSCU', 'LBXSSE', 'LBXWBCSI', 'LBDHDD', 'LBDSSESI', 'URDFLOW1', 'WTSAF2YR', 'LBXSPH', 'WTSOG2YR', 'URXUMA', 'LBXSASSI', 'LBXEOPCT', 'LBDSGBSI', 'LBXSAPSI', 'LBXTC', 'LBXMC', 'LBXTHG', 'LBDSCHSI', 'URXMOH', 'LBDSALSI', 'PHAFSTHR', 'DXDLAPF', 'DXDTRBMD', 'DXDSTBMC', 'DXXTRFAT', 'DXXHEFAT', 'DXDSTLE', 'DXXRALI', 'DXDTOFAT', 'DXDTOBMD', 'DXXLSBMC', 'DXXTRLI', 'DXDRATOT', 'BPXDI3', 'DXXHEA', 'DXDTOPF', 'DXDRLTOT', 'DXDTRPF', 'DXXLLBMD', 'DXXRLLI', 'BMXARMC', 'DXDRALE', 'DXXLLBMC', 'DXXLALI', 'DXXLRBMC', 'DXXPEA', 'DXDRAPF', 'BPXDI1', 'DXXPEBMD', 'DXDTRTOT', 'DXXRAA', 'DXDLLPF', 'BMXARML', 'DXXHELI', 'DXXRRA', 'DXDSTBMD', 'BPXDI4', 'DXXLLLI', 'DXDRLPF', 'BPXDI2', 'DXXLSBMD', 'BPXPLS', 'DXDTOBMC', 'DXDTOLE', 'DXXLRBMD', 'DXDTRLE', 'DXXHEBMC', 'DXDSTTOT', 'DXXRLA', 'BMXLEG', 'BPXSY1', 'DXXLAA', 'DXXLABMC', 'DXDLALE', 'DXDTRA', 'BMXWT', 'DXXTSBMC', 'DXXLLA', 'DXXLLFAT', 'DXDTOLI', 'BMXHT', 'DXXLABMD', 'DXDSTLI', 'DXXRRBMD', 'DXXLSA', 'DXDSTA', 'DXDTRBMC', 'DXXRABMC', 'DXDHELE', 'DXDHETOT', 'DXXHEBMD', 'DXDTOA', 'DXXRABMD', 'DXDRLLE', 'DXXLRA', 'DXXRRBMC', 'BPXSY2', 'DXDLLLE', 'DXXTSBMD', 'BMXBMI', 'DXDSTFAT', 'DXXRLFAT', 'DXXRLBMD', 'DXXRAFAT', 'DXDLATOT', 'BMXWAIST', 'DXDLLTOT', 'DXDSTPF', 'DXDHEPF', 'DXXLAFAT', 'DXXRLBMC', 'BPXSY3', 'BPXSY4', 'DXXTSA', 'DXXPEBMC', 'DXDTOTOT']
print(len(cont_vars))

dis_vars = ['DMDYRSUS', 'DMDEDUC2', 'DMDHHSIZ', 'DMDFMSIZ', 'BPXML1', 'BPACSZ', 'FCX10DI', 'FCX11DI', 'FCX06DI', 'FCX08DI', 'FCX07DI', 'FCX09DI', 'LBXBGE', 'URXVOL3', 'URXUAS5', 'URDFLOW3', 'LBXVTE', 'LBDSTBSI', 'URXUMN', 'LBXVTC', 'LBXSTB', 'PHAANTMN', 'PHAALCMN', 'PHACOFMN', 'PHASUPMN', 'URDTIME3', 'LBXV2A', 'LBXV1D', 'LBXVMC', 'LBXVNB', 'LBXV3B', 'LBXVCB', 'LBXVCT', 'LBDMONO', 'LBDEONO', 'LBDBANO', 'PHASUPHR', 'LBXSCLSI', 'LBXSNASI', 'PHAGUMHR', 'PHAANTHR', 'PHACOFHR', 'PHAALCHR', 'LBXSC3SI']
print(len(dis_vars))

cat_vars = ['RIDRETH1', 'RIDEXMON', 'DMDCITZN', 'DMDMARTL', 'RIDEXPRG', 'URDMNPLC', 'PHQ060', 'ORXH51', 'ORXH64', 'ORXHPC', 'LBDHEG', 'ORXGL', 'ORXH69', 'PHQ050', 'LBDVCTLC', 'LBD2DFLC', 'LBXHCR', 'ORXH62', 'URDCNPLC', 'URDUSNLC', 'ORXHPV', 'LBDVTCLC', 'URDUA3LC', 'URDUTLLC', 'URDUTULC', 'URDUDALC', 'ORXH11', 'URDUCOLC', 'ORXH73', 'URXUTRI', 'ORXH26', 'ORXH31', 'LBDHBG', 'ORXH82', 'LBXHE1', 'ORXH53', 'ORXH58', 'URDUURLC', 'LBDHEM', 'URDMZPLC', 'URDUBALC', 'LBDVTELC', 'LBDV1DLC', 'PHQ020', 'LBDV3BLC', 'URDUACLC', 'URDUSBLC', 'ORXH35', 'ORXH84', 'LBDIHGLC', 'URXPREG', 'LBDV4CLC', 'PHQ040', 'URDUPBLC', 'LBDBGELC', 'URDUMMAL', 'URDUMNLC', 'ORXH68', 'URDUA5LC', 'URDMEPLC', 'ORXH81', 'LBDVOXLC', 'ORXGH', 'LBDVCBLC', 'LBDVBZLC', 'ORXH52', 'URDMBPLC', 'ORXH56', 'LBXHE2', 'ORXH61', 'LBDBGMLC', 'URDECPLC', 'URDUCDLC', 'LBDVNBLC', 'URDMOHLC', 'ORXH33', 'ORXH06', 'ORXHPI', 'LBDTHGLC', 'URDUMOLC', 'LBDVXYLC', 'URDMHPLC', 'URDCOPLC', 'URDMC1LC', 'ORXH45', 'ORXH55', 'ORXH71', 'PHQ030', 'LBDVMCLC', 'URDUSRLC', 'ORXH40', 'ORXH39', 'ORXH83', 'ORXH66', 'LBXHBC', 'LBDVDBLC', 'LBDVEBLC', 'ORXH72', 'ORXH54', 'ORXH70', 'URDUCSLC', 'URXUCL', 'LBDV2ALC', 'LBDWFLLC', 'URDUABLC', 'LBXHA', 'ORXH59', 'ORXH67', 'ORXH42', 'ORXH18', 'URDMHHLC', 'PHDSESN', 'URDMIBLC', 'ORXH16', 'LBXHBS', 'LBXHCG', 'OHARNF', 'OHAPOS', 'BMIARML', 'OHDDESTS', 'OHAROCGP', 'OHX23TC', 'BPAEN2', 'OHAROCOH', 'OHAROCDE', 'OHX02TC', 'OHX30TC', 'OHXIMP', 'OHX14TC', 'OHAROTH', 'BMDSTATS', 'OHX16TC', 'OHX05TC', 'BMIARMC', 'BPAEN3', 'OHX26TC', 'OHAREC', 'BPAARM', 'OHAROCCI', 'BMILEG', 'BPAEN4', 'OHX01TC', 'BMXRECUM', 'OHX09TC', 'BMIHT', 'BPXCHR', 'OHX32TC', 'OHX18TC', 'OHDRCSTS', 'BPXPTY', 'BMXHEAD', 'OHX19TC', 'OHX15TC', 'OHDEXSTS', 'OHAROCDT', 'BMIHEAD', 'BMIRECUM', 'OHX03TC', 'BPXPULS', 'OHX17TC', 'BMIWT', 'BPAEN1', 'OHX31TC', 'OHX08TC', 'BMIWAIST', 'DXARLBV', 'DXALLBV', 'DXARABV', 'FCX10DI', 'OHX06TC', 'DXALABV', 'DXARLTV', 'OHX04TC', 'OHX12TC', 'FCX08DI', 'OHX27TC', 'OHX13TC', 'DXAHEBV', 'OHX22TC', 'OHX10TC', 'OHX29TC', 'OHX21TC', 'FCX07DI', 'OHX28TC', 'OHX07TC', 'FCX09DI', 'DXARATV', 'OHX24TC', 'PEASCCT1', 'DXALLTV', 'OHX25TC', 'OHX11TC', 'DXAHETV', 'DXAEXSTS']      
print(len(cat_vars))

224
44
194


## 6. Final dataset

In [220]:
HC_df = df.groupby(['HSD010','Class'])['Class'].count().unstack()
HC_df['Percentage_0'] = df[df.Class == 0].groupby(['HSD010'])['HSD010'].count()/df[df.Class==0].shape[0]
HC_df['Percentage_1'] = df[df.Class == 1].groupby(['HSD010'])['HSD010'].count()/df[df.Class==1].shape[0]

HC_df

Class,0,1,Percentage_0,Percentage_1
HSD010,,,,
1.0,596.0,22.0,0.062187,0.038128
2.0,1941.0,147.0,0.202525,0.254766
3.0,3310.0,197.0,0.345367,0.341421
4.0,2098.0,111.0,0.218907,0.192374
5.0,436.0,42.0,0.045492,0.072790
9.0,2.0,NaN,0.000209,NaN


In [221]:
df = df[df.HSD010 < 4]

df.shape, df[df.Class == 1].shape, df[df.Class == 0].shape

((6213, 799), (366, 799), (5847, 799))

## 7. Check missing data of all features

In [222]:
all_vars = cont_vars+dis_vars
miss_df = pd.DataFrame(df[all_vars].isnull().sum(),columns=['Count'])
miss_df['Percentage'] = 100 * df[all_vars].isnull().sum()/len(df)
miss_df = miss_df.sort_values('Percentage', ascending=True)
miss_df = miss_df.reset_index()
miss_df.columns = ['Feature','Count','Percentage']
miss_df.head(20)

,Feature,Count,Percentage
0,RIDAGEYR,0,0.000000
1,DMDEDUC2,0,0.000000
2,DMDHHSIZ,0,0.000000
3,DMDFMSIZ,0,0.000000
4,PHAFSTHR,52,0.836955
5,PHAFSTMN,52,0.836955
6,BMXWT,53,0.853050
7,BMXHT,54,0.869145
8,BMXBMI,64,1.030098
9,URXCRS,110,1.770481


https://pypi.org/project/mixed-naive-bayes/

# Part 3: Data Preprocessing

## 1. Cont vars

### 1.1. Handling missing values

In [223]:
miss_cont_df = pd.DataFrame(df[cont_vars].isnull().sum(),columns=['Count'])
miss_cont_df['Percentage'] = 100 * df[cont_vars].isnull().sum()/len(df)
miss_cont_df = miss_cont_df.sort_values('Percentage', ascending=True)
miss_cont_df.head()

,Count,Percentage
RIDAGEYR,0,0.000000
PHAFSTMN,52,0.836955
PHAFSTHR,52,0.836955
BMXWT,53,0.853050
BMXHT,54,0.869145


#### Select cont vars with missing value <= 30%

In [224]:
cont_vars = list(miss_cont_df[miss_cont_df.Percentage <= 20].index)
print(len(cont_vars))
print(cont_vars)

78
['RIDAGEYR', 'PHAFSTMN', 'PHAFSTHR', 'BMXWT', 'BMXHT', 'BMXBMI', 'URXUMA', 'URXCRS', 'URXUMS', 'BPXPLS', 'BMXARML', 'BMXARMC', 'LBXMC', 'LBXMPSI', 'LBXRBCSI', 'LBXPLTSI', 'LBXHGB', 'LBXMCHSI', 'LBXWBCSI', 'LBXRDW', 'LBXMCVSI', 'LBXEOPCT', 'LBXMOPCT', 'LBDNENO', 'LBDLYMNO', 'LBXBAPCT', 'LBXNEPCT', 'LBXLYPCT', 'BMXWAIST', 'BMXLEG', 'LBXTC', 'LBDTCSI', 'LBDHDD', 'LBDHDDSI', 'LBXSAL', 'LBDSALSI', 'LBDSCRSI', 'LBXSGL', 'LBDSGLSI', 'LBXSCR', 'LBXSGB', 'LBXSBU', 'LBDSGBSI', 'LBXSPH', 'LBXSTP', 'LBDSPHSI', 'LBXSOSSI', 'LBDSTPSI', 'LBDSBUSI', 'LBXSAPSI', 'LBXSKSI', 'LBXSGTSI', 'LBDSCHSI', 'LBXSCH', 'LBDSUASI', 'LBXSUA', 'LBDSCASI', 'LBXSCA', 'LBXSTR', 'LBDSTRSI', 'LBXSIR', 'LBDSIRSI', 'LBXSATSI', 'LBXSASSI', 'LBXSLDSI', 'BPXDI2', 'BPXSY2', 'BPXSY3', 'BPXDI3', 'BPXSY1', 'BPXDI1', 'INDFMPIR', 'LBDBCDSI', 'LBDTHGSI', 'LBXBPB', 'LBXBCD', 'LBDBPBSI', 'LBXTHG']


In [225]:
import scipy

ttest_df = pd.DataFrame(columns = ['Feature','FeatureName','t-stats','p-value','Skew','Kurtosis'])
ttest_df['Feature'] = cont_vars
ttest_df['FeatureName'] = cont_vars

df0 = df[df['Class'] == 0]
df1 = df[df['Class'] == 1]

for var in cont_vars:
    result = scipy.stats.ranksums(df0[df0[var].isna()==False][var], df1[df1[var].isna()==False][var])
    ttest_df.loc[ttest_df['Feature'] == var,'t-stats'] = result[0]
    ttest_df.loc[ttest_df['Feature'] == var,'p-value'] = result[1]
    ttest_df.loc[ttest_df['Feature'] == var,'Skew'] = df[var].skew()
    ttest_df.loc[ttest_df['Feature'] == var,'Kurtosis'] = df[var].kurt()
  
ttest_df['abs_tstats'] = np.abs(ttest_df['t-stats'])
ttest_df = ttest_df.sort_values(['abs_tstats'], ascending = False)
ttest_df = ttest_df.merge(miss_df, left_on = 'Feature',right_on='Feature',how='inner')

ttest_df

,Feature,FeatureName,t-stats,p-value,Skew,Kurtosis,abs_tstats,Count,Percentage
0,RIDAGEYR,RIDAGEYR,-10.9384,7.5559e-28,0.228539,-1.07904,10.9384,0,0.000000
1,LBXLYPCT,LBXLYPCT,6.47063,9.75925e-11,0.420741,0.666245,6.47063,209,3.363914
2,LBDLYMNO,LBDLYMNO,6.27825,3.42408e-10,20.3985,979.567,6.27825,209,3.363914
3,LBDSTPSI,LBDSTPSI,5.45576,4.87639e-08,0.41769,1.29181,5.45576,308,4.957347
4,LBXSTP,LBXSTP,5.45576,4.87639e-08,0.41769,1.29181,5.45576,308,4.957347
5,LBXSBU,LBXSBU,-4.61495,3.93181e-06,2.15124,12.1316,4.61495,308,4.957347
6,LBDSBUSI,LBDSBUSI,-4.61495,3.93181e-06,2.15117,12.1301,4.61495,308,4.957347
7,LBXNEPCT,LBXNEPCT,-4.35048,1.35842e-05,-0.254389,0.573341,4.35048,209,3.363914
8,LBXSOSSI,LBXSOSSI,-4.20697,2.58818e-05,-0.281232,3.1387,4.20697,308,4.957347
9,LBXSUA,LBXSUA,-3.68582,0.000227966,0.760137,1.07861,3.68582,311,5.005633


In [226]:
sig_cont_vars = list(ttest_df[ttest_df['p-value'] <= 0.05].sort_values(['abs_tstats'],ascending=False).Feature)
print(sig_cont_vars)
print(len(sig_cont_vars))

['RIDAGEYR', 'LBXLYPCT', 'LBDLYMNO', 'LBDSTPSI', 'LBXSTP', 'LBXSBU', 'LBDSBUSI', 'LBXNEPCT', 'LBXSOSSI', 'LBXSUA', 'LBDSUASI', 'LBXMOPCT', 'LBXPLTSI', 'BPXDI2', 'LBXSGB', 'LBDSGBSI', 'BPXDI1', 'BMXARML', 'LBXEOPCT', 'LBDSCRSI', 'LBXSCR', 'LBXSAL', 'LBDSALSI', 'LBDSGLSI', 'LBXSGL', 'BPXDI3', 'LBXSKSI', 'LBXMCVSI', 'LBXSTR', 'LBDSTRSI', 'INDFMPIR']
31


#### Remove observations with 40% of missing data

In [227]:
df['FeatureCount'] = df[sig_cont_vars].count(axis=1)
df['FeatureMissing'] = len(sig_cont_vars) - df['FeatureCount']
df['MissingPercentage'] = df.FeatureMissing/len(sig_cont_vars)
df[['FeatureMissing','MissingPercentage']].describe()

,FeatureMissing,MissingPercentage
count,6213.000000,6213.000000
mean,1.453726,0.046894
std,4.981097,0.160681
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,1.000000,0.032258
max,29.000000,0.935484


In [228]:
print(df[(df['MissingPercentage'] > 0.6) & (df['Class'] == 1)].shape)
df = df[df['MissingPercentage'] <= 0.6]
df = df.reset_index(drop=True)

df.shape, df[df.Class == 1].shape, df[df.Class == 0].shape

(26, 802)


((5997, 802), (340, 802), (5657, 802))

#### Filling missing data 

In [229]:
for var in cont_vars:
    df[var] = df[var].fillna(df[var].median())

In [230]:
from sklearn.naive_bayes import GaussianNB

X_train, X_test, y_train, y_test = train_test_split(df[cont_vars], df['Class'], test_size=0.25, random_state=1)

def NaiveBayesPrediction(X_train, y_train, X_test, y_test):
    clf = GaussianNB()
    clf.fit(X_train, y_train)
    
    y_pred_gnb = clf.predict(X_test)
    y_prob_pred_gnb = clf.predict_proba(X_test)
    # how did our model perform?
    count_misclassified = (y_test != y_pred_gnb).sum()
    
    print("GaussianNB")
    print("=" * 30)
    print('Misclassified samples: {}'.format(count_misclassified))
    accuracy = accuracy_score(y_test, y_pred_gnb)
    print('Accuracy: {:.2f}'.format(accuracy))
    
    print("Recall score : ", recall_score(y_test, y_pred_gnb , average='micro'))
    print("Precision score : ",precision_score(y_test, y_pred_gnb , average='micro'))
    print("F1 score : ",f1_score(y_test, y_pred_gnb , average='micro'))
    
    print(classification_report(y_test, y_pred_gnb))
    
NaiveBayesPrediction(X_train, y_train, X_test, y_test)

GaussianNB
Misclassified samples: 202
Accuracy: 0.87
Recall score :  0.8653333333333333
Precision score :  0.8653333333333333
F1 score :  0.8653333333333333
              precision    recall  f1-score   support

           0       0.95      0.90      0.93      1413
           1       0.14      0.25      0.18        87

    accuracy                           0.87      1500
   macro avg       0.54      0.58      0.55      1500
weighted avg       0.90      0.87      0.88      1500



In [231]:
from sklearn.naive_bayes import GaussianNB 
model = GaussianNB()

kfold = KFold(n_splits = 10, random_state = 10)
scores = cross_val_score(model,X_train,y_train,cv=kfold,scoring='recall_macro')
print(scores)

print("Accuracy: %0.2f%% (+/- %0.2f%%)" % (100*scores.mean(), 100*scores.std()))

[0.5962371  0.60776488 0.5070922  0.53633373 0.53777619 0.52464789
 0.5630282  0.6018367  0.52610837 0.5714818 ]
Accuracy: 55.72% (+/- 3.41%)


### 1.2. Remove outliers

In [232]:
for var in cont_vars:
    upper_boundary, lower_boundary = find_skewed_boundaries(df, var, 3.5)
    df.loc[df[var] <= lower_boundary,var] = lower_boundary
    df.loc[df[var] >= upper_boundary,var] = upper_boundary

## 8. Feature scaling

In [233]:
from sklearn.naive_bayes import GaussianNB

X_train, X_test, y_train, y_test = train_test_split(df[cont_vars], df['Class'], test_size=0.25, random_state=1)
    
NaiveBayesPrediction(X_train, y_train, X_test, y_test)

GaussianNB
Misclassified samples: 196
Accuracy: 0.87
Recall score :  0.8693333333333333
Precision score :  0.8693333333333333
F1 score :  0.8693333333333333
              precision    recall  f1-score   support

           0       0.95      0.91      0.93      1413
           1       0.14      0.25      0.18        87

    accuracy                           0.87      1500
   macro avg       0.55      0.58      0.56      1500
weighted avg       0.90      0.87      0.89      1500



In [234]:
from sklearn.naive_bayes import GaussianNB 
model = GaussianNB()

kfold = KFold(n_splits = 10, random_state = 10)
scores = cross_val_score(model,X_train,y_train,cv=kfold,scoring='recall_macro')
print(scores)

print("Accuracy: %0.2f%% (+/- %0.2f%%)" % (100*scores.mean(), 100*scores.std()))

[0.56776557 0.54780479 0.53703704 0.55427632 0.57774157 0.53169014
 0.51837898 0.60065466 0.48805419 0.57850756]
Accuracy: 55.02% (+/- 3.13%)


### 1.3. Gaussian Transformation - Type 2

In [235]:
df1 = df.copy()

Gauss_transformed_vars = ['URDTIME1','URXVOL1','LBXTHG','LBDTHGSI','LBXSGTSI','URXCRS','URXUMS','URXUMA']

In [236]:
from sklearn.preprocessing import quantile_transform

for var in Gauss_transformed_vars:
    df1[var] = quantile_transform(np.array(df1[var]).reshape(-1,1), n_quantiles=20, random_state=0, copy=True)

In [237]:
X_train, X_test, y_train, y_test = train_test_split(df1[cont_vars], df1['Class'], test_size=0.25, random_state=1)

NaiveBayesPrediction(X_train, y_train, X_test, y_test)

GaussianNB
Misclassified samples: 200
Accuracy: 0.87
Recall score :  0.8666666666666667
Precision score :  0.8666666666666667
F1 score :  0.8666666666666667
              precision    recall  f1-score   support

           0       0.95      0.90      0.93      1413
           1       0.14      0.26      0.19        87

    accuracy                           0.87      1500
   macro avg       0.55      0.58      0.56      1500
weighted avg       0.91      0.87      0.88      1500



In [238]:
from sklearn.naive_bayes import GaussianNB 
model = GaussianNB()

kfold = KFold(n_splits = 10, random_state = 10)
scores = cross_val_score(model,X_train,y_train,cv=kfold,scoring='recall_macro')
print(scores)

print("Accuracy: %0.2f%% (+/- %0.2f%%)" % (100*scores.mean(), 100*scores.std()))

[0.61538462 0.58272859 0.53703704 0.54949163 0.57422869 0.53286385
 0.53894715 0.6222495  0.48210181 0.60006387]
Accuracy: 56.35% (+/- 4.13%)


### 1.4. Only use sig cont vars 

In [239]:
len(sig_cont_vars)

31

In [240]:
X_train, X_test, y_train, y_test = train_test_split(df[sig_cont_vars], df['Class'], test_size=0.25, random_state=1)
    
NaiveBayesPrediction(X_train, y_train, X_test, y_test)

GaussianNB
Misclassified samples: 181
Accuracy: 0.88
Recall score :  0.8793333333333333
Precision score :  0.8793333333333333
F1 score :  0.8793333333333333
              precision    recall  f1-score   support

           0       0.95      0.92      0.93      1413
           1       0.14      0.21      0.17        87

    accuracy                           0.88      1500
   macro avg       0.54      0.56      0.55      1500
weighted avg       0.90      0.88      0.89      1500



In [241]:
from sklearn.naive_bayes import GaussianNB 
model = GaussianNB()

kfold = KFold(n_splits = 10, random_state = 10)
scores = cross_val_score(model,X_train,y_train,cv=kfold,scoring='recall_macro')
print(scores)

print("Accuracy: %0.2f%% (+/- %0.2f%%)" % (100*scores.mean(), 100*scores.std()))

[0.52597403 0.58590348 0.5106383  0.55061304 0.54128907 0.54107981
 0.53125955 0.58969813 0.49995895 0.57499468]
Accuracy: 54.51% (+/- 2.90%)


### 1.5. Remove features with high correlation

In [242]:
# build a dataframe with the correlation between features
# remember that the absolute value of the correlation
# coefficient is important and not the sign

corrmat = df[sig_cont_vars].corr()
corrmat = corrmat.abs().unstack() # absolute value of corr coef
corrmat = corrmat.sort_values(ascending=False)

corrmat = pd.DataFrame(corrmat).reset_index()
corrmat.columns = ['feature1', 'feature2', 'corr']
corrmat['MissingF1'] = corrmat.feature1.apply(lambda x:MissingPercentage(x))
corrmat['MissingF2'] = corrmat.feature2.apply(lambda x:MissingPercentage(x))
corrmat.head()

,feature1,feature2,corr,MissingF1,MissingF2
0,LBXSGB,LBDSGBSI,1.0,0.0,0.0
1,LBDSGBSI,LBXSGB,1.0,0.0,0.0
2,INDFMPIR,INDFMPIR,1.0,0.0,0.0
3,LBXMCVSI,LBXMCVSI,1.0,0.0,0.0
4,LBXSKSI,LBXSKSI,1.0,0.0,0.0


In [243]:
correlated_groups = corrmat[corrmat['corr'] > 0.90]
correlated_groups

,feature1,feature2,corr,MissingF1,MissingF2
0,LBXSGB,LBDSGBSI,1.000000,0.0,0.0
1,LBDSGBSI,LBXSGB,1.000000,0.0,0.0
2,INDFMPIR,INDFMPIR,1.000000,0.0,0.0
3,LBXMCVSI,LBXMCVSI,1.000000,0.0,0.0
4,LBXSKSI,LBXSKSI,1.000000,0.0,0.0
5,BPXDI3,BPXDI3,1.000000,0.0,0.0
6,LBXSGL,LBXSGL,1.000000,0.0,0.0
7,LBDSGLSI,LBDSGLSI,1.000000,0.0,0.0
8,LBDSALSI,LBDSALSI,1.000000,0.0,0.0
9,LBXSAL,LBXSAL,1.000000,0.0,0.0


In [244]:
sig_cont_vars = []
remaining_vars = correlated_groups.feature1.unique()

while(len(remaining_vars) > 0):
    feature = remaining_vars[0]
    correlated_block = correlated_groups[correlated_groups.feature1 == feature]
    min_ind = correlated_block[['MissingF2']].idxmin() 
    sel_var = correlated_block.feature2[min_ind].values[0]
    removed_vars = [var for var in list(correlated_block.feature2.values)]
    remaining_vars = [var for var in remaining_vars if var not in removed_vars]
    if sel_var not in sig_cont_vars:
        sig_cont_vars = sig_cont_vars + [sel_var]    
    
print(sig_cont_vars)
len(sig_cont_vars)

['LBDSGBSI', 'INDFMPIR', 'LBXMCVSI', 'LBXSKSI', 'BPXDI3', 'LBXSGL', 'LBDSALSI', 'LBXSCR', 'LBXEOPCT', 'BMXARML', 'BPXDI1', 'LBDSTRSI', 'BPXDI2', 'LBXPLTSI', 'LBXMOPCT', 'LBDSUASI', 'LBXSOSSI', 'LBXNEPCT', 'LBDSBUSI', 'LBXSTP', 'LBDLYMNO', 'RIDAGEYR']


22

In [245]:
X_train, X_test, y_train, y_test = train_test_split(df[sig_cont_vars], df['Class'], test_size=0.25, random_state=1)
    
NaiveBayesPrediction(X_train, y_train, X_test, y_test)

GaussianNB
Misclassified samples: 136
Accuracy: 0.91
Recall score :  0.9093333333333333
Precision score :  0.9093333333333333
F1 score :  0.9093333333333333
              precision    recall  f1-score   support

           0       0.95      0.95      0.95      1413
           1       0.19      0.17      0.18        87

    accuracy                           0.91      1500
   macro avg       0.57      0.56      0.57      1500
weighted avg       0.91      0.91      0.91      1500



In [246]:
from sklearn.naive_bayes import GaussianNB 
model = GaussianNB()

kfold = KFold(n_splits = 10, random_state = 10)
scores = cross_val_score(model,X_train,y_train,cv=kfold,scoring='recall_macro')
print(scores)

print("Accuracy: %0.2f%% (+/- %0.2f%%)" % (100*scores.mean(), 100*scores.std()))

[0.47435897 0.54118287 0.51221434 0.52287679 0.51537522 0.4903169
 0.54999491 0.57369522 0.49462233 0.54593357]
Accuracy: 52.21% (+/- 2.93%)


In [247]:
from mixed_naive_bayes import MixedNB

X_train, X_test, y_train, y_test = train_test_split(df[sig_cont_vars], df['Class'], test_size=0.25, random_state=1)

def MixedBayesPrediction(X_train, y_train, X_test, y_test):
    clf = MixedNB()
    clf.fit(X_train, y_train)
    
    y_pred_gnb = clf.predict(X_test)
    y_prob_pred_gnb = clf.predict_proba(X_test)
    # how did our model perform?
    count_misclassified = (y_test != y_pred_gnb).sum()
    
    print("GaussianNB")
    print("=" * 30)
    print('Misclassified samples: {}'.format(count_misclassified))
    accuracy = accuracy_score(y_test, y_pred_gnb)
    print('Accuracy: {:.2f}'.format(accuracy))
    
    print("Recall score : ", recall_score(y_test, y_pred_gnb , average='micro'))
    print("Precision score : ",precision_score(y_test, y_pred_gnb , average='micro'))
    print("F1 score : ",f1_score(y_test, y_pred_gnb , average='micro'))
    
    print(classification_report(y_test, y_pred_gnb))
    
MixedBayesPrediction(X_train, y_train, X_test, y_test)

GaussianNB
Misclassified samples: 136
Accuracy: 0.91
Recall score :  0.9093333333333333
Precision score :  0.9093333333333333
F1 score :  0.9093333333333333
              precision    recall  f1-score   support

           0       0.95      0.95      0.95      1413
           1       0.19      0.17      0.18        87

    accuracy                           0.91      1500
   macro avg       0.57      0.56      0.57      1500
weighted avg       0.91      0.91      0.91      1500

